GOALS: 

      CNN-LSTM/Spatial-Temporial LSTM https://towardsdatascience.com/spatial-temporal-convlstm-for-crash-prediction-411909ed2cfa
      Figure out how to break data into depth, lat and lon


#Getting Data

In [2]:
import requests 
file_url = "https://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/ssh/data.nc"
	
r = requests.get(file_url, stream = True) 

with open("./data.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

import os
size = os.path.getsize('./data.nc') 
print('Size of file is', size, 'bytes')

Size of file is 1573874020 bytes


### SSH

In [1]:
import xarray as xr
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

/Users/jkenglish/Downloads/anaconda3/envs/tensorflow/lib/python3.7/site-packages/xarray/backends/cfgrib_.py:28: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  "Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. "


In [3]:
ds_ssh = xr.open_dataset("./data.nc" , decode_times=False)
#ds_ssh = xr.open_dataset("http://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/ssh/dods" , decode_times=False)

/Users/jkenglish/Downloads/anaconda3/envs/tensorflow/lib/python3.7/site-packages/xarray/backends/plugins.py:68: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [4]:
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_ssh = fix_calendar(ds_ssh)
ds_ssh = xr.decode_cf(ds_ssh)
ds_ssh

<xarray.Dataset>
Dimensions:  (lon: 720, lat: 330, time: 1656)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 ...

In [5]:
ds_ssh

<xarray.Dataset>
Dimensions:  (lon: 720, lat: 330, time: 1656)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 ...

In [6]:
ds_ssh = ds_ssh.where(((ds_ssh.lon > 20 ) & (ds_ssh.lon < 80 ) & (ds_ssh.lat > 10) & (ds_ssh.lat <20)), drop=True)

In [7]:
ds_ssh

<xarray.Dataset>
Dimensions:  (time: 1656, lat: 20, lon: 120)
Coordinates:
  * lon      (lon) float32 20.25 20.75 21.25 21.75 ... 78.25 78.75 79.25 79.75
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Data variables:
    ssh      (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

In [8]:
ds_ssh['ssh']

<xarray.DataArray 'ssh' (time: 1656, lat: 20, lon: 120)>
array([[[       nan,        nan,        nan, ...,        nan,
         0.47368455, 0.4705949 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47322145],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.45891967],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.49529132, 0.49132225],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.4916404 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.47177067],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
         0.552477  , 0.5522664 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.5573779 ],
        [       nan,        nan,        nan, ...,        nan,
                nan, 0.54433674],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 20.25 20.75 21.25 21.75 ... 78.25 78.75 79.25 79.75
  * lat      (lat) float32 10.25 10.75 11.25 11.75 ... 18.25 18.75 19.25 19.75
  * time     (time) object 1871-01-16 00:00:00 ... 2008-12-16 00:00:00
Attributes:
    pointwidth:     1.0
    units:          meters
    standard_name:  sea_surface_height
    long_name:      SEA SURFACE HEIGHT

In [9]:
np.array(ds_ssh['ssh']).shape

(1656, 20, 120)

In [10]:
np.array(ds_ssh['ssh'][1650][0])

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan, -0.01835824,  0.02075754,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,  0.02848718,  0.0399644 ,  0.09675951,  0.13464183,
        0.15300493,  0.20257978,  0.28198448,  0.38239264,  0.47

### Salt

In [7]:
import requests 
file_url = "https://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/.salt/time/(1872)/VALUES/data.nc"
	
r = requests.get(file_url, stream = True) 

with open("./data1872.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

import os
size = os.path.getsize('./data1872.nc') 
print('Size of file is', size, 'bytes')
	
r = requests.get(file_url, stream = True) 

with open("./data1872.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

import os
size = os.path.getsize('./data1872.nc') 
print('Size of file is', size, 'bytes')

Size of file is 456197252 bytes
Size of file is 456197252 bytes


In [8]:
ds_salt = xr.open_dataset("./data1872.nc" , decode_times=False)

In [15]:
def fix_calendar(ds, timevar='time'):
    if ds[timevar].attrs['calendar'] == '360':
        ds[timevar].attrs['calendar'] = '360_day'
    return ds

ds_salt = fix_calendar(ds_salt)
ds_salt = xr.decode_cf(ds_salt)
ds_salt

<xarray.Dataset>
Dimensions:  (lon: 720, lat: 330, depth: 40, time: 12)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * depth    (depth) float32 5.01 15.07 25.28 ... 4.875e+03 5.125e+03 5.375e+03
  * time     (time) object 1872-01-16 00:00:00 ... 1872-12-16 00:00:00
Data variables:
    salt     (time, depth, lat, lon) float32 ...

In [16]:
ds_salt.keys()

KeysView(<xarray.Dataset>
Dimensions:  (lon: 720, lat: 330, depth: 40, time: 12)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * depth    (depth) float32 5.01 15.07 25.28 ... 4.875e+03 5.125e+03 5.375e+03
  * time     (time) object 1872-01-16 00:00:00 ... 1872-12-16 00:00:00
Data variables:
    salt     (time, depth, lat, lon) float32 ...)

In [26]:
#ds_salt = xr.open_dataarray(url, chunks={'S': 1, 'L': 1})

ds_salt_sub = ds_salt.where(((ds_salt.lon==0.75 )&(ds_salt.lat==88.25) & (ds_salt.depth==5.01) ), drop=True)

In [27]:
ds_salt_sub

<xarray.Dataset>
Dimensions:  (time: 12, depth: 1, lat: 1, lon: 1)
Coordinates:
  * lon      (lon) float32 0.75
  * lat      (lat) float32 88.25
  * depth    (depth) float32 5.01
  * time     (time) object 1872-01-16 00:00:00 ... 1872-12-16 00:00:00
Data variables:
    salt     (time, depth, lat, lon) float32 31.54 31.42 31.48 ... 31.5 31.52

In [28]:

ds_salt_sub['salt']

<xarray.DataArray 'salt' (time: 12, depth: 1, lat: 1, lon: 1)>
array([[[[31.538584]]],


       [[[31.421606]]],


       [[[31.475916]]],


       [[[31.640408]]],


       [[[31.74586 ]]],


       [[[31.712181]]],


       [[[31.651155]]],


       [[[31.515562]]],


       [[[31.566011]]],


       [[[31.583931]]],


       [[[31.498869]]],


       [[[31.519655]]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 0.75
  * lat      (lat) float32 88.25
  * depth    (depth) float32 5.01
  * time     (time) object 1872-01-16 00:00:00 ... 1872-12-16 00:00:00
Attributes:
    pointwidth:  1.0
    units:       g/kg
    long_name:   SALINITY

In [29]:
np.array(ds_salt_sub['salt'])

array([[[[31.538584]]],


       [[[31.421606]]],


       [[[31.475916]]],


       [[[31.640408]]],


       [[[31.74586 ]]],


       [[[31.712181]]],


       [[[31.651155]]],


       [[[31.515562]]],


       [[[31.566011]]],


       [[[31.583931]]],


       [[[31.498869]]],


       [[[31.519655]]]], dtype=float32)

In [30]:
np.array(ds_salt_sub['salt']).shape

(12, 1, 1, 1)

In [31]:

np.array(ds_salt['salt']).shape

(12, 40, 330, 720)

In [32]:
np.array(ds_salt['salt'])

array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [31.499346, 31.538584, 31.58072 , ..., 31.496075, 31.49632 ,
          31.49658 ],
         [31.372227, 31.371088, 31.369877, ..., 31.375214, 31.374289,
          31.373293],
         [31.160454, 31.160376, 31.160292, ..., 31.151   , 31.157486,
          31.160528]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [31.693327, 31.693806, 31.693897, ..., 31.694862, 31.694355,
          31.693811],
         [31.389193, 31.387829, 31.38638 , ..., 31

### Temperatures

In [33]:
import requests 
file_url = "https://iridl.ldeo.columbia.edu/SOURCES/.CARTON-GIESE/.SODA/.v2p2p4/.temp/time/(1872)/VALUES/data.nc"
	
r = requests.get(file_url, stream = True) 

with open("./data1872temp.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

import os
size = os.path.getsize('./data1872temp.nc') 
print('Size of file is', size, 'bytes')
	
r = requests.get(file_url, stream = True) 

with open("./data1872temp.nc", "wb") as file: 
	for block in r.iter_content(chunk_size = 1024): 
		if block: 
			file.write(block) 

import os
size = os.path.getsize('./data1872temp.nc') 
print('Size of file is', size, 'bytes')

Size of file is 456197320 bytes
Size of file is 456197320 bytes


In [34]:
ds_temp = xr.open_dataset("./data1872temp.nc" , decode_times=False)

In [35]:
ds_temp.keys()

KeysView(<xarray.Dataset>
Dimensions:  (lon: 720, lat: 330, depth: 40, time: 12)
Coordinates:
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 88.25 88.75 89.25
  * depth    (depth) float32 5.01 15.07 25.28 ... 4.875e+03 5.125e+03 5.375e+03
  * time     (time) float32 -1.056e+03 -1.054e+03 ... -1.046e+03 -1.044e+03
Data variables:
    temp     (time, depth, lat, lon) float32 ...)

In [ ]:
ds_temp_sub = ds_temp.where(((ds_temp.lon > 50 )&(ds_temp.lat<70) & (ds_temp.depth<90) ), drop=True)

In [ ]:
ds_temp_sub

<xarray.Dataset>
Dimensions:  (depth: 8, lat: 291, lon: 620, time: 1656)
Coordinates:
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 68.75 69.25 69.75
  * lon      (lon) float32 50.25 50.75 51.25 51.75 ... 358.2 358.8 359.2 359.8
  * depth    (depth) float32 5.01 15.07 25.28 35.76 46.61 57.98 70.02 82.92
  * time     (time) float32 -1.068e+03 -1.066e+03 -1.066e+03 ... 586.5 587.5
Data variables:
    temp     (time, depth, lat, lon) float32 dask.array<chunksize=(1024, 8, 291, 620), meta=np.ndarray>
Attributes:
    Conventions:  IRIDL

In [ ]:
ds_temp_sub['temp']

<xarray.DataArray 'temp' (time: 1656, depth: 8, lat: 291, lon: 620)>
dask.array<where, shape=(1656, 8, 291, 620), dtype=float32, chunksize=(1024, 8, 291, 620), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -75.25 -74.75 -74.25 -73.75 ... 68.75 69.25 69.75
  * lon      (lon) float32 50.25 50.75 51.25 51.75 ... 358.2 358.8 359.2 359.8
  * depth    (depth) float32 5.01 15.07 25.28 35.76 46.61 57.98 70.02 82.92
  * time     (time) float32 -1.068e+03 -1.066e+03 -1.066e+03 ... 586.5 587.5
Attributes:
    pointwidth:     1.0
    long_name:      TEMPERATURE
    standard_name:  sea_water_temperature
    units:          Celsius_scale

In [ ]:
np.array(ds_temp_sub['lat']).shape

(291,)

# Data Manpulation

In [ ]:
!pip install netCDF4

In [ ]:
ds = nc.Dataset('data.nc')

#Data Visulazation

In [ ]:
!pip install cartopy

In [ ]:
import cartopy.crs as ccrs

In [ ]:
days = np.arange(0,365)
# creating meshgrid from our list of spatial coordinates
lon, lat = np.meshgrid(longs,lats)
# generate a frame for each day
for i in days:
    # set the axes' spatial projection, styling, and title
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_title('Precipitation for day '+ str(i+1))
    
    # generate the contour map
    plt.contourf(longs, lats, tp[i,], 60,
    transform=ccrs.PlateCarree())
    
    # save the figure and give a pause for visualization
    plt.savefig(f'./output/{i+1}.jpg')
    plt.pause(1)
plt.show()

In [ ]:
import PIL
image_frames = []
days = np.arange(20,80) # sub-sample day number 20 to 80
for k in days:
    new_frame = PIL.Image.open(f'./output/{k}.jpg') 
    image_frames.append(new_frame)
# save as GIF
image_frames[0].save('tp_timelapse.gif',
                     format='GIF',
                     append_images = image_frames[1: ],
                     save_all = True, 
                     duration = 100,
                     loop = 0)